**1) Загрузите данные MovieLens**

In [1]:
import pandas as pd

In [2]:
!wget 'https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx' -O MovieLens.zip

--2023-05-13 19:09:01--  https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.google.com (drive.google.com)... 142.251.2.102, 142.251.2.139, 142.251.2.101, ...
Connecting to drive.google.com (drive.google.com)|142.251.2.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/3p302tvpubm6dgkh02gi1kk8jbm81pva/1684004925000/02611596255248067438/*/1m0rwReR09achL0xTM6QPoN4tykz5bOMx?uuid=873afa58-e1de-4746-87f2-65725e03eb0a [following]
--2023-05-13 19:09:03--  https://doc-0g-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/3p302tvpubm6dgkh02gi1kk8jbm81pva/1684004925000/02611596255248067438/*/1m0rwReR09achL0xTM6QPoN4tykz5bOMx?uuid=873afa58-e1de-4746-87f2-65725e03eb0a
Resolving doc-0g-84-docs.googleusercontent.com (doc-0g-84-docs.googleusercontent.com)... 142.250.141.132, 2607:f8b0:4023:c0b::84
Connecting to doc-0g-84

In [3]:
!unzip MovieLens.zip

Archive:  MovieLens.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: tags.csv                


Нам понадобиться только таблица с рейтингами, поэтому берем только ее

In [4]:
ratings = pd.read_csv('ratings.csv')

In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
dataset = ratings.drop('timestamp', axis = 1)

In [7]:
dataset.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


**2) Используя любые модели из пакета *Surprise*, получить RMSE на тестовом сете 0,87 и ниже**

In [8]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095468 sha256=e666dc995ce1b6796b8d7a3a1fe2ee457d07acb16e357341b247314cce01779f
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [9]:
from surprise import SVDpp, Dataset, Reader
from surprise.model_selection import cross_validate

Смотрим и минимальный и максимальный рейтинг, чтобы задать параметры в Reader 

In [10]:
ratings.rating.min()

0.5

In [11]:
ratings.rating.max()

5.0

In [12]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

**Используем алгоритм SVD++ (Singular Value Decomposition ++)** 

-это алгоритм рекомендательной системы, который использует метод сингулярного разложения матриц (Singular Value Decomposition, SVD) для выявления скрытых признаков элементов и пользователей в матрице рейтингов.

Одним из основных недостатков базового алгоритма SVD является то, что он не учитывает информацию о действиях пользователей и о элементах, которые пользователь уже оценил. SVD++ решает эту проблему, учитывая также информацию об отклонении каждого пользователя от среднего рейтинга и о том, насколько каждый элемент отличается от среднего рейтинга.

Алгоритм SVD++ представляет собой расширение базового алгоритма SVD и включает в себя дополнительные матрицы для учета информации о действиях пользователей и элементах. В частности, SVD++ использует следующие матрицы:

* Матрица сходства между элементами. Эта матрица позволяет учитывать информацию о схожести элементов и увеличивать точность предсказания рейтингов для элементов, которые пользователь еще не оценил.
* Матрица сходства между пользователями. Эта матрица позволяет учитывать информацию о схожести пользователей и увеличивать точность предсказания рейтингов для пользователя, который еще не оценил некоторые элементы.
* Матрица смещения пользователей. Эта матрица позволяет учитывать информацию об отклонении каждого пользователя от среднего рейтинга.
* Матрица смещения элементов. Эта матрица позволяет учитывать информацию о том, насколько каждый элемент отличается от среднего рейтинга.

SVD++ использует эти матрицы для вычисления взвешенной комбинации схожести элементов, схожести пользователей, смещения пользователей и смещения элементов, которая используется для предсказания рейтингов для элементов, которые пользователь еще не оценил.



$$\hat{r}_{ui} = \mu + b_u + b_i + q_i^T\left(p_u + |I_u|^{-\frac{1}{2}} \sum_{j \in I_u}y_j\right)$$

где:
- $\hat{r}_{ui}$ - предсказанный рейтинг пользователя $u$ для элемента $i$
- $\mu$ - средний рейтинг по всем элементам и пользователям
- $b_u$ - смещение пользователя $u$ относительно среднего рейтинга
- $b_i$ - смещение элемента $i$ относительно среднего рейтинга
- $q_i$ - вектор скрытых признаков элемента $i$
- $p_u$ - вектор скрытых признаков пользователя $u$
- $I_u$ - множество элементов, которые пользователь $u$ оценил
- $y_j$ - вектор скрытых признаков элемента $j$

Для обучения модели SVD++ используется метод стохастического градиентного спуска (SGD), а целевая функция определяется как сумма квадратов отклонений предсказанных рейтингов от реальных рейтингов:

$$\text{minimize } \sum_{(u,i) \in R_{train}}(r_{ui} - \hat{r}_{ui})^2 + \lambda\left(b_i^2 + b_u^2 + ||q_i||^2 + ||p_u||^2 + \sum_{j \in I_u}||y_j||^2\right)$$

где:
- $R_{train}$ - множество пар (пользователь, элемент), для которых имеются реальные рейтинги
- $\lambda$ - коэффициент регуляризации

Для оптимизации целевой функции используется стохастический градиентный спуск с адаптивным шагом обучения (Adaptive Learning Rate, Adagrad).

In [13]:
algo = SVDpp()

In [14]:
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8574  0.8589  0.8624  0.8643  0.8651  0.8616  0.0030  
Fit time          102.09  100.17  95.80   97.28   98.08   98.69   2.21    
Test time         11.53   10.75   11.84   11.61   11.82   11.51   0.40    


{'test_rmse': array([0.85738575, 0.85893401, 0.86237644, 0.86425737, 0.8651182 ]),
 'fit_time': (102.09365701675415,
  100.17070269584656,
  95.79887175559998,
  97.28483319282532,
  98.07758951187134),
 'test_time': (11.532340288162231,
  10.748648881912231,
  11.83526086807251,
  11.61470103263855,
  11.82033085823059)}

Получим рекомедации для пользователя 1

In [62]:
movie_user1 = dataset[dataset['userId']==1]['movieId'].tolist()# id фильмов которые пользователь уже посмотрел

In [63]:
movie_all = dataset['movieId'].tolist()# id всех фильмов

In [64]:
movie_not_user1 = list(set(movie_all) - set(movie_user1))# id фильмов которые пользователь не смотрел

In [75]:
recommend_user1 = pd.DataFrame()

In [76]:
# делаем предсказание по каждому фильму который не смотрел пользователь
# и добавляем в датафрейм
for i in movie_not_user1:
  mov_rating = algo.predict(uid=1.0, iid = i).est
  row = {'movieId':i, 'ratings': mov_rating}
  recommend_user1 = pd.concat([recommend_user1, pd.DataFrame([row])])

Для наглядности добавим названия фильмов

In [68]:
movies = pd.read_csv('movies.csv')

In [72]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [77]:
recommend_user1 = recommend_user1.merge(movies[['title','movieId']], on = 'movieId')

Получаем 10 первых фильмов, которым бы пользователь 1 поставил бы самые высокие оценки

In [78]:
recommend_user1.sort_values('ratings', ascending=False).head(10)

,movieId,ratings,title
967,1225,5.0,Amadeus (1984)
974,1233,5.0,"Boot, Das (Boat, The) (1981)"
6519,142488,5.0,Spotlight (2015)
963,1217,5.0,Ran (1985)
7083,48516,5.0,"Departed, The (2006)"
965,1221,5.0,"Godfather: Part II, The (1974)"
783,951,5.0,His Girl Friday (1940)
966,1223,5.0,"Grand Day Out with Wallace and Gromit, A (1989)"
285,318,5.0,"Shawshank Redemption, The (1994)"
2715,3451,5.0,Guess Who's Coming to Dinner (1967)
